In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
from bs4 import BeautifulSoup as bs
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
from src.etl import extract as ex

In [2]:
load_dotenv()

True

In [3]:
RUTA_SERVICE = os.getenv("RUTA_SERVICE")
URL_ESCRAPEO_INICIAL = os.getenv("URL_ESCRAPEO_INICIAL")

In [4]:
url_destinos= URL_ESCRAPEO_INICIAL #url destinos totales
sopa_destinos = ex.crear_sopa(url_destinos) #creamos la sopa
#Extraemos todos los contenidos de la clase landing-destination-title
clase_landing_destination_title=sopa_destinos.find_all("div", {"class":"landing-destination-title"})
print(f'Existen {len(clase_landing_destination_title)} destinos diferentes')

Existen 123 destinos diferentes


In [5]:
df_destinos_totales = ex.extracción_destinos_con_url(sopa_destinos) #escrapeo los destinos totales
df_destinos_totales.head()

,nombre_pais,num_viajes_ofertados,url_destino,fecha_escrapeo
0,Albania,1,https://es.tui.com/viajes/albania,2025-05-02
1,Alemania,11,https://es.tui.com/viajes/alemania,2025-05-02
2,Antigua,2,https://es.tui.com/viajes/antiguacaribe,2025-05-02
3,Arabia Saudí,1,https://es.tui.com/viajes/arabia-saudi,2025-05-02
4,Argentina,21,https://es.tui.com/viajes/argentina,2025-05-02


In [6]:
df_destinos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   nombre_pais           123 non-null    object        
 1   num_viajes_ofertados  123 non-null    int64         
 2   url_destino           123 non-null    object        
 3   fecha_escrapeo        123 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 4.0+ KB


In [7]:
#añado los destinos actuales a los destinos anteriores y guardo el DF con los destinos totales acumulados
ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS): #comprueba si existe un fichero ya con destinos
    df_existente_destinos = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #en caso de existir lo importa
    df_combinado_destinos = pd.concat([df_existente_destinos,df_destinos_totales], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_destinos=df_destinos_totales #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_destinos["en_ultimo_escrapeo"] = df_combinado_destinos["nombre_pais"].apply(lambda x: ex.disponible_ultimo_escrapeo(x, df_destinos_totales, 'nombre_pais'))
df_combinado_destinos= df_combinado_destinos.drop_duplicates()
df_combinado_destinos.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #guarda el fichero en formato formato pickle

In [8]:
df_combinado_destinos.shape

(2758, 5)

In [9]:
df_combinado_destinos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758 entries, 0 to 2757
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   nombre_pais           2758 non-null   object        
 1   num_viajes_ofertados  2758 non-null   int64         
 2   url_destino           2758 non-null   object        
 3   fecha_escrapeo        2758 non-null   datetime64[ns]
 4   en_ultimo_escrapeo    2758 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 107.9+ KB


In [10]:
df_combinado_destinos.en_ultimo_escrapeo.value_counts()

en_ultimo_escrapeo
Si    2758
Name: count, dtype: int64

In [11]:
df_viajes_totales_destinos, df_continentes, df_opciones = ex.continentes_viajes_totales_destinos_url_y_opciones_1(df_destinos_totales)

 Para Albania en Europa se ha encontrado 1 viajes diferentes
 Para Alemania en Europa se ha encontrado 11 viajes diferentes
 Para Antigua en Caribe se ha encontrado 2 viajes diferentes
 Para Arabia Saudí en Oriente Medio se ha encontrado 1 viajes diferentes
 Para Argentina en América se ha encontrado 21 viajes diferentes
 Para Armenia en Oriente Medio se ha encontrado 2 viajes diferentes
 Para Aruba en Caribe se ha encontrado 3 viajes diferentes
 Para Australia en Oceanía se ha encontrado 16 viajes diferentes
 Para Austria en Europa se ha encontrado 14 viajes diferentes
 Para Bahamas en Caribe se ha encontrado 5 viajes diferentes
 Para Barbados en Caribe se ha encontrado 2 viajes diferentes
 Para Bélgica en Europa se ha encontrado 5 viajes diferentes
 Para Belice en América se ha encontrado 1 viajes diferentes
 Para Bolivia en América se ha encontrado 2 viajes diferentes
 Para Botsuana en África se ha encontrado 8 viajes diferentes
 Para Brasil en América se ha encontrado 9 viajes dife

In [12]:
df_viajes_totales_destinos.head()

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url,fecha_escrapeo
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-05-02
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-05-02
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-05-02
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-05-02
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-05-02


In [13]:
df_viajes_totales_destinos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            1071 non-null   object        
 1   nombre_viaje    1071 non-null   object        
 2   duracion_viaje  1071 non-null   object        
 3   itinerario      1071 non-null   object        
 4   precio          1071 non-null   int64         
 5   url             1071 non-null   object        
 6   fecha_escrapeo  1071 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 58.7+ KB


In [14]:
#añado los viajes actuales a los viajes anteriores y guardo el DF con los viajes totales de todos los destinos acumulados
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_VIAJES")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES): #comprueba si existe un fichero ya con viajes
    df_existente_viajes_totales= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES)#en caso de existir lo importa
    df_combinado_viajes_totales = pd.concat([df_existente_viajes_totales,df_viajes_totales_destinos], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_viajes_totales=df_viajes_totales_destinos  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_viajes_totales["en_ultimo_escrapeo"] = df_combinado_viajes_totales["url"].apply(lambda x: ex.disponible_ultimo_escrapeo(x, df_viajes_totales_destinos, 'url'))
df_combinado_viajes_totales =df_combinado_viajes_totales.drop_duplicates()
df_combinado_viajes_totales.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES) #guarda el fichero en formato pickle

In [15]:
df_combinado_viajes_totales.en_ultimo_escrapeo.value_counts()

en_ultimo_escrapeo
Si    23519
No      582
Name: count, dtype: int64

In [16]:
df_combinado_viajes_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24101 entries, 0 to 24100
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                24101 non-null  object        
 1   nombre_viaje        24101 non-null  object        
 2   duracion_viaje      24101 non-null  object        
 3   itinerario          24101 non-null  object        
 4   precio              24101 non-null  int64         
 5   url                 24079 non-null  object        
 6   fecha_escrapeo      24101 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  24101 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.5+ MB


In [17]:
df_continentes.head()

,continente,pais,fecha_escrapeo
0,Europa,Albania,2025-05-02
1,Europa,Alemania,2025-05-02
2,Caribe,Antigua,2025-05-02
3,Oriente Medio,Arabia Saudí,2025-05-02
4,América,Argentina,2025-05-02


In [18]:
df_continentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   continente      123 non-null    object        
 1   pais            123 non-null    object        
 2   fecha_escrapeo  123 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.0+ KB


In [19]:
#añado los continentes actuales a los continentes anteriores y guardo el DF con los continentes totales de todos los destinos acumulados
ARCHIVO_GUARDAR_CONTINENTES = os.getenv("ARCHIVO_GUARDAR_CONTINENTES")
if os.path.exists(ARCHIVO_GUARDAR_CONTINENTES): #comprueba si existe un fichero ya con continentes
    df_existente_continentes= pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES)#en caso de existir lo importa
    df_combinado_continentes = pd.concat([df_existente_continentes,df_continentes], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_continentes=df_continentes  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_continentes["en_ultimo_escrapeo"] = df_combinado_continentes["continente"].apply(lambda x: ex.disponible_ultimo_escrapeo(x, df_continentes, 'continente'))
df_combinado_continentes.drop_duplicates(inplace=True)
df_combinado_continentes.to_pickle(ARCHIVO_GUARDAR_CONTINENTES) #guarda el fichero en formato pickle

In [20]:
df_combinado_continentes.en_ultimo_escrapeo.value_counts()

en_ultimo_escrapeo
Si    1926
Name: count, dtype: int64

In [21]:
df_combinado_continentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1926 entries, 0 to 1925
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   continente          1926 non-null   object        
 1   pais                1926 non-null   object        
 2   fecha_escrapeo      1926 non-null   datetime64[ns]
 3   en_ultimo_escrapeo  1926 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 60.3+ KB


In [22]:
df_opciones.head()

,pais,nombre_viaje,opcion,precio,url_opcion,fecha_escrapeo
0,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Madrid,1835,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-05-02
1,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Barcelona,1930,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-05-02
2,Alemania,Encantos del Rin y Valle del Mosela,Salida desde Bilbao,1952,https://es.tui.com/viaje/encantos-del-rin-y-va...,2025-05-02
3,Alemania,El Rin Romántico y la Alsacia,Salida desde Madrid,1945,https://es.tui.com/viaje/el-rin-romantico-y-la...,2025-05-02
4,Alemania,El Rin Romántico y la Alsacia,Salida desde Barcelona,2040,https://es.tui.com/viaje/el-rin-romantico-y-la...,2025-05-02


In [23]:
df_opciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844 entries, 0 to 843
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pais            844 non-null    object        
 1   nombre_viaje    844 non-null    object        
 2   opcion          844 non-null    object        
 3   precio          844 non-null    int64         
 4   url_opcion      844 non-null    object        
 5   fecha_escrapeo  844 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 39.7+ KB


In [24]:
#añado las opciones actuales a las opciones anteriores y guardo el DF con las opciones totales de todos los destinos acumulados
ARCHIVO_GUARDAR_OPCIONES_VIAJES = os.getenv("ARCHIVO_GUARDAR_OPCIONES_VIAJES")
if os.path.exists(ARCHIVO_GUARDAR_OPCIONES_VIAJES): #comprueba si existe un fichero ya con continentes
    df_existente_opciones= pd.read_pickle(ARCHIVO_GUARDAR_OPCIONES_VIAJES)#en caso de existir lo importa
    df_combinado_opciones_viajes = pd.concat([df_existente_opciones,df_opciones], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_opciones_viajes=df_opciones  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_opciones_viajes["en_ultimo_escrapeo"] = df_combinado_opciones_viajes["url_opcion"].apply(lambda x: ex.disponible_ultimo_escrapeo(x, df_opciones, 'url_opcion'))
df_combinado_opciones_viajes= df_combinado_opciones_viajes.drop_duplicates()
df_combinado_opciones_viajes.to_pickle(ARCHIVO_GUARDAR_OPCIONES_VIAJES) #guarda el fichero en formato pickle

In [25]:
df_combinado_opciones_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12363 entries, 0 to 12362
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                12363 non-null  object        
 1   nombre_viaje        12363 non-null  object        
 2   opcion              12363 non-null  object        
 3   precio              12363 non-null  int64         
 4   url_opcion          12359 non-null  object        
 5   fecha_escrapeo      12363 non-null  datetime64[ns]
 6   en_ultimo_escrapeo  12363 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 676.2+ KB


In [26]:
df_combinado_opciones_viajes.en_ultimo_escrapeo.value_counts()

en_ultimo_escrapeo
Si    12028
No      335
Name: count, dtype: int64

In [27]:
df_combinado_opciones_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12363 entries, 0 to 12362
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                12363 non-null  object        
 1   nombre_viaje        12363 non-null  object        
 2   opcion              12363 non-null  object        
 3   precio              12363 non-null  int64         
 4   url_opcion          12359 non-null  object        
 5   fecha_escrapeo      12363 non-null  datetime64[ns]
 6   en_ultimo_escrapeo  12363 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 676.2+ KB
